In [1]:
import pandas as pd
import numpy as np

# Đường dẫn đến file feather cần đọc
file_path = 'GQUIC_small/dataset1/Train/train_Transfer_64.feather'

# Đọc file feather
df = pd.read_feather(file_path)

In [2]:
# Chọn cột '0' đến '127' cho mỗi flow_id và chuyển thành một danh sách các ma trận
matrices = []
labels = []
for flow_id, group in df.groupby('flow_id'):
    matrix = group.loc[:, '0':'63'].values
    label = group['Label'].values[0] 
    matrices.append(matrix)
    labels.append(label)

# Chuyển danh sách ma trận thành mảng NumPy
X = np.array(matrices)
X = X / 255
y = np.array(labels)

In [3]:
# import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Khởi tạo mô hình
model = keras.Sequential()

# Thêm lớp Convolutional Layer với 32 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(20, 64, 1), padding='same'))

# Thêm lớp Convolutional Layer với 32 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))

# Thêm lớp Max Pooling 2D để giảm kích thước đầu vào đi 2 lần
model.add(layers.MaxPooling2D((2, 2)))

# Thêm lớp Dropout để giảm hiện tượng overfitting
model.add(layers.Dropout(0.5))

# Thêm lớp Convolutional Layer với 64 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))

# Thêm lớp Convolutional Layer với 64 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))

# Thêm lớp Max Pooling 2D để giảm kích thước đầu vào đi 2 lần
model.add(layers.MaxPooling2D((2, 2)))

# Thêm lớp Flatten để làm phẳng đầu ra của lớp trước khi đi vào các lớp fully connected
model.add(layers.Flatten())

# Thêm lớp fully connected (Dense Layer) với 64 units và hàm kích hoạt ReLU
model.add(layers.Dense(64, activation='relu'))

# Thêm lớp Dropout để giảm hiện tượng overfitting
model.add(layers.Dropout(0.5))

# Thêm lớp fully connected (Dense Layer) cuối cùng với số lớp đầu ra phụ thuộc vào bài toán của bạn
model.add(layers.Dense(5, activation='softmax'))

# In thông tin mô hình
model.summary()

# Biên dịch mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 20, 64, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 64, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 10, 32, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 10, 32, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 32, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 32, 64)        36928     
                                                        

In [4]:
model.fit(X, y, epochs=10, validation_data=(X, y))

Epoch 1/10
412/412 [==============================] - 55s 130ms/step - loss: 1.0276 - accuracy: 0.5716 - val_loss: 0.5353 - val_accuracy: 0.8310
Epoch 2/10
412/412 [==============================] - 53s 129ms/step - loss: 0.6916 - accuracy: 0.7146 - val_loss: 0.4738 - val_accuracy: 0.8451
Epoch 3/10
412/412 [==============================] - 53s 129ms/step - loss: 0.5974 - accuracy: 0.7620 - val_loss: 0.3801 - val_accuracy: 0.8604
Epoch 4/10
412/412 [==============================] - 53s 128ms/step - loss: 0.5439 - accuracy: 0.7837 - val_loss: 0.3638 - val_accuracy: 0.8602
Epoch 5/10
412/412 [==============================] - 54s 130ms/step - loss: 0.5092 - accuracy: 0.7963 - val_loss: 0.3385 - val_accuracy: 0.8657
Epoch 6/10
412/412 [==============================] - 62s 152ms/step - loss: 0.4811 - accuracy: 0.8052 - val_loss: 0.2973 - val_accuracy: 0.8773
Epoch 7/10
412/412 [==============================] - 61s 148ms/step - loss: 0.4707 - accuracy: 0.8107 - val_loss: 0.3102 - val_ac

In [5]:
# Đường dẫn đến file feather cần đọc
file_path_test = 'GQUIC_small/dataset1/Test/test_Transfer_64.feather'

# Đọc file feather
df_test = pd.read_feather(file_path)

In [6]:
# Chọn cột '0' đến '127' cho mỗi flow_id và chuyển thành một danh sách các ma trận
matrices_test = []
labels_test = []
for flow_id, group in df.groupby('flow_id'):
    matrix = group.loc[:, '0':'63'].values
    label = group['Label'].values[0] 
    matrices_test.append(matrix)
    labels_test.append(label)

# Chuyển danh sách ma trận thành mảng NumPy
X_test = np.array(matrices_test)
X_test = X_test / 255
y_test = np.array(labels_test)

In [10]:
# Đánh giá mô hình trên tập kiểm tra
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Độ chính xác trên tập kiểm tra:", test_accuracy * 100)

# Dự đoán các nhãn trên tập kiểm tra
y_pred = model.predict(X_test)

# Chuyển đổi dự đoán thành nhãn dự đoán (lớp dự đoán)
y_pred_classes = np.argmax(y_pred, axis=1)

# Hiển thị ma trận nhầm lẫn
from sklearn.metrics import confusion_matrix, classification_report
confusion = confusion_matrix(y_test, y_pred_classes)
print("Ma trận nhầm lẫn:")
print(confusion)

# Hiển thị báo cáo phân loại
class_report = classification_report(y_test, y_pred_classes)
print("Báo cáo phân loại:")
print(class_report)


412/412 [==============================] - 19s 46ms/step - loss: 0.2667 - accuracy: 0.8976
Độ chính xác trên tập kiểm tra: 89.76150751113892
412/412 [==============================] - 19s 46ms/step
Ma trận nhầm lẫn:
[[2842    0    0    0    0]
 [   0 1848  161    9   16]
 [   6   49 2459    2  292]
 [   0  392   16 1330    2]
 [  23   56  324    0 3339]]
Báo cáo phân loại:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2842
           1       0.79      0.91      0.84      2034
           2       0.83      0.88      0.85      2808
           3       0.99      0.76      0.86      1740
           4       0.92      0.89      0.90      3742

    accuracy                           0.90     13166
   macro avg       0.90      0.89      0.89     13166
weighted avg       0.90      0.90      0.90     13166



In [8]:
# Đếm số lần xuất hiện của mỗi giá trị trong y_pred_classes
counts = np.bincount(y_pred_classes)

# Hiển thị kết quả
for class_idx, count in enumerate(counts):
    print(f"Số lần xuất hiện của lớp {class_idx}: {count}")

Số lần xuất hiện của lớp 0: 2871
Số lần xuất hiện của lớp 1: 2345
Số lần xuất hiện của lớp 2: 2960
Số lần xuất hiện của lớp 3: 1341
Số lần xuất hiện của lớp 4: 3649
